# Let's try the ChefGPT

Let's try our newly trained model and see how well it performs

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_10k_v1",device_map="auto")
tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_10k_v1")

In [2]:
#
generation_kwargs_v2 = {
    "max_length": 512,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
#     "temperature": 0.7,
    "top_p": 0.95,
    "num_beams": 5,
    "num_return_sequences": 3,
    "early_stopping": True    
}

#
generation_kwargs_v3 = {
    "max_length": 512,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3
}

generation_kwargs_v4 = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

def generate_recipe_v1(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)    
    return output_text

def generate_recipe_v2(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).input_ids
    output_ids = model.generate(input_ids, **generation_kwargs_v2)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

def generate_recipe_v3(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).input_ids
    output_ids = model.generate(input_ids, **generation_kwargs_v3)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

def generate_recipe_v4(prompt):
    input_text = f"generate recipe: {prompt}"
    inputs = tokenizer(
        input_text, 
        max_length=512, 
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids, 
        attention_mask=attention_mask,
        **generation_kwargs_v4
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [3]:
# food_items = "butter, mushrooms, garlic, zucchini, onion, eggs, Ricotta cheese, shredded Monterey Jack, frozen spinach, dill weed, salt"
# food_items = "chicken breasts, lime marinade, tomatoes, shredded cheese, tortilla chips"
#food_items = "pie shell, eggs, butter, sugar, cocoa, flour, milk, vanilla"
# food_items = "cooking oats, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "cooking oat, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "coconut, sugar, bran cereal, pumpkin seeds, flour, rolled oats, milk"
# food_items = "Salt, Pepper, FILLING, Olive Oil, Sausage, Fresh Spinach Leaves, Eggs, Egg Whites, Coconut Milk, Colby Cheese, Salt, Pepper, Hot Sauce, Green Onions"
food_items = "chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix"

recipe = generate_recipe_v1(food_items)
# s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(recipe)
# print(s)

/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


title: Chicken Soup ingredients: 1 lb. chicken, 1 c. peas, 1 c. carrots, 1 can cream of chicken soup, 1 can cream of celery soup, 1 can chicken broth, 1 c. milk, 1 c. Bisquick mix directions: Cook chicken in boiling water until tender. Drain. Add peas carrots cream of chicken soup cream of celery soup chicken broth milk Bisquick mix and mix well. Add chicken broth and milk. Stir well. Add Bisquick mix and mix well.


In [5]:
# very CPU intensive!
recipe = generate_recipe_v2(food_items)
# s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(recipe)
# sprint(s)

title: Chicken Casserole ingredients: 1 lb. chicken, 1 c. chopped peas, 2 carrots, chopped, 1 can cream of chicken soup, 2 cans cream of celery soup, 3 cans chicken broth, 1 cup milk, Bisquick mix directions: Preheat oven to 400u00b0F. In a large bowl combine chicken soup milk and milk. Mix well. Pour over chicken. Cover and refrigerate until ready to serve.


In [4]:
recipe = generate_recipe_v3(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(s)

/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


title: Chicken Frozen 

Ingredients: 1 lb. chicken breasted breasted breasted breasted breast, 1 lb. peas, 4 carrots, 3 (6 oz.) cans cream of chicken soup, 2 cans cream of celery soup (with no fat, no fat, no fat), 1 (10 oz.) can chicken broth, 1 (10 oz.) can milk, 1/2 cup Bisquick mix 

Directions: For chicken and carrots reheat oven to 400u00b0F. Cook for about 15 minutes at 350u00b0F. Stir chicken cream of chicken soup soup juice of remaining sauce and the remaining oil over high heat stirring occasionally until the chicken is tender about 8 minutes. Combine broth milk milk Bisquick mix and chicken soup. Pour into an 8 x 8-inch pan and bake 30 minutes for 1 hour longer. Serves 7.


In [5]:
recipe = generate_recipe_v4(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
# print(recipe)
print(s)

title: Chicken Breast Pot Pie 

Ingredients: 6 lbs boneless boneless chicken, peas, diced, 2 carrots, mashed, 2 cans cream of chicken soup, shredded, 1 can cream of celery soup, 1 chicken broth, 1 1/2 cups liquid, 2 cups evaporated milk, Bisquick mix 

Directions: Cook chicken in boiling water until tender about 10 minutes; drain well. Mix chicken pea potatoes carrot soup cream of mushroom soup and chicken broth. Add 2 cups water. Add Bisquicker mix to chicken. Cover and refrigerate 1 hour or until chicken is tender enough to bake overnight.


## Using Pipelines

In [8]:
from transformers import pipeline
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_v3")
# tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_v3", model_max_len=512)

# model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_5K_v3")
# tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_5K_v3")

# model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_10K_v1")
# tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_10K_v1")

generator = pipeline(model=model, tokenizer=tokenizer, task="text2text-generation")

In [9]:
# food_items = 'cooking oats, brown sugar, salad oil, eggs, salt, almond extract'
# food_items = 'ground beef, vinegar, sugar, chili powder, Worcestershire sauce, catsup, mustard, onion'
# food_items = "cooking oat, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "coconut, sugar, bran cereal, pumpkin seeds, flour, rolled oats, milk"
# food_items = "butter, mushrooms, garlic, zucchini, onion, eggs, Ricotta cheese, shredded Monterey Jack, frozen spinach, dill weed, salt"
# food_items = "cooking oats, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "Salt, Pepper, FILLING, Olive Oil, Sausage, Fresh Spinach Leaves, Eggs, Egg Whites, Coconut Milk, Colby Cheese, Salt, Pepper, Hot Sauce, Green Onions"
food_items = "chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix"

tokenizer_kwargs = {
    "max_length": 512,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95,
    "num_return_sequences": 2
}

response = generator(f"generate recipe: {food_items}", **tokenizer_kwargs)

print(response[0]['generated_text'].replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:"))

recipe: Tossed chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix, liquid stir together cream of chicken broth and milk, cream of cream of meat soup 

Ingredients: 1/2 cup chicken breast, 1/2 cup milk, 1/2 cup Bisquick mix. Sprinkle over chicken soup, salt, and pepper. Cover and bring to a boil. Bake for 40 minutes. Remove chicken from heat and place. Heat remaining cream of chicken soup into a 12-ounce plastic container. Cover and leave to cool for 30 minutes, and stir in water. Add milk and milk. Add salt and pepper. Top with pepper.
